In [ ]:
# 导入对应的 package
import numpy as np
import pandas as pd
import requests
import bs4
import time
import json
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
font_hei = matplotlib.font_manager.FontProperties(
    fname=r"SimHei.ttf", size=15)  # 设置中文字体

In [ ]:
def get_data(base_url, page):
    """
    获取B站单个页面的视频信息
    base_url: json的api
    page: 页数
    return 一个二维的list,16列
    """
    url = base_url.format(str(page))  # 得到对应的url
    response = requests.get(url, headers=header)  # 发出请求，获取回应
    videos = pd.read_json(response.text[37:-1]).iloc[:, 3].iloc[0]  # 获取列表
    videos_list = []  # 用来储存数据
    for video in videos:  # 循环每个番剧
        video_inf = []  # 用来储存每个番剧的信息
        video_inf.extend(video['stat']['aid'])  # av号
        video_inf.extend(video["title"])  # 标题
        video_inf.extend(video['owner']["name"])  # 上传者
        video_inf.extend(time.ctime(video['pubdate']))  # 上传时间

        video_inf.extend(video['stat']['view'])  # 观看人数
        video_inf.extend(video['stat']['danmaku'])  # 弹幕数
        video_inf.extend(video['stat']['reply'])  # 评论数量
        video_inf.extend(video['stat']['favorite'])  # 收藏数量
        video_inf.extend(video['stat']['coin'])  # 投币数
        video_inf.extend(video['stat']['share'])  # 分享数
        video_inf.extend(video['stat']['like'])  # 点喜欢的人数
        video_inf.extend(video['stat']['dislike'])  # 点不喜欢的人数
        video_inf.extend(video['stat']["now_rank"])  # 排名
        video_inf.extend(video['stat']["his_rank"])  # 排名

        video_inf.extend(video['tname'])  # 标签
        video_inf.extend(video['desc'])  # 描述

        videos_list.append(video_inf)
    return videos_list


def get_all_data(n):
    """
    n 为页数，从第一页爬到第 n 页
    返回的是一个 DataFrame
    """
    # 数据对应的列名
    col = ['av号', '标题', 'up主', '上传日期', '观看次数', '弹幕', 'reply',
           '收藏', '投币', '分享', '喜欢', '不喜欢', '现在排名', '他的排名', '标签', '描述']
    # 建立 DataFrame 来储存数据
    data = pd.DataFrame(columns=col)

    # 循环页数
    for i in range(1, n + 1):
        data_temp = pd.DataFrame(
            get_data(base_url, i), columns=col)  # 对应页的数据，临时储存
        data = pd.concat([data, data_temp], axis=0,
                         ignore_index=True)  # 与之前的数据合并在一块
        # 适当的停顿，防止请求过快，封ip
        time.sleep(np.random.rand() * np.random.randint(1, 3))
    return data

In [ ]:
# 番剧对应的 url,通过chorme的 inspect 查找得到
base_url = "https://api.bilibili.com/x/web-interface/newlist?callback=jqueryCallback_bili_4431985766884974&rid=33&type=0&pn={}&ps=20&jsonp=jsonp&_=1558156016281"
# 一些的请求头，伪装成浏览器，可以防止一些反爬虫的机制
header = {"Referer": "https://www.bilibili.com/v/anime/serial/?spm_id_from=333.334.b_7072696d6172795f6d656e75.8",
          "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"}

In [ ]:
data = get_all_data(157)  # 获取1327页的视频数据,页数为网页直接查看获得
# data.to_csv('bilibili_anime_data.csv') # 保存数据，因没有设置index=False, 导致后面读取数据时 多了一列-_-

In [ ]:
# 数据清洗

In [ ]:
# 首先查看下数据是否有 nan 值
data = pd.read_csv('bilibili_anime_data.csv')
data.isnull().sum(

In [ ]:
# 因为描述不重要，所以只查看up主是 nan 的那一行
data[data.isnull()['up主']]

In [ ]:
# 去掉这一行
data.drop(26397, inplace=True)

In [ ]:
# 再次确认缺失值
data.isnull().sum()

In [ ]:
# 查看数据有没有重复项
data.duplicated().sum()

In [ ]:
# 确认下最小值
data.min()

In [ ]:
# 将上传时间 转换成 datetime 类型
data['上传日期'] = pd.to_datetime(data['上传日期'])

In [ ]:
# 查看观看次数是-1的番剧是哪些
data[-1 == data['观看次数']]

In [ ]:
data.drop(data[-1 == data['观看次数']].index, inplace=True)

In [ ]:
# 再次确认一下
data.min()

In [ ]:
# 因为保存的时候，没有将 index 设置为 False, 所以多了个一列,现在删除这一列
data.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
# 数据分析

In [ ]:
# 查看一下统计
data.describe()

In [ ]:
# 看一下各数据的相关性
data.corr()

In [ ]:

# 查看up主投稿的数量
data.groupby("up主").count().sort_values("av号", ascending=False)  # 根据up主投稿的数量排序

In [ ]:
data.sort_values("收藏", ascending=False).reset_index(drop=True)  # 收藏数量排序

In [ ]:
data.sort_values("观看次数", ascending=False).reset_index(drop=True)